<a href="https://colab.research.google.com/github/Mahmoud-Ibrahim-Mamrstein/Spectroscopic-constants-from-atomic-properties/blob/main/networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Runtime / Change runtime type


SyntaxError: ignored

In [ ]:
pip install networkx[all]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install graphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pydot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import sklearn as sk
from matplotlib.backends.backend_pdf import PdfPages
import re
import seaborn as sns
from matplotlib import pyplot
%matplotlib inline
import time
import math
from math import sqrt
import scipy
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold, LeaveOneOut
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Product, Matern, WhiteKernel, RBF, DotProduct, ExpSineSquared
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
import networkx as nx
import plotly.graph_objects as go
import graphviz
import pydot

In [ ]:
k=np.load('kernel.npy')
k

array([[1.        , 0.99985405, 0.99845343, ..., 0.95776585, 0.96426798,
        0.9946027 ],
       [0.99985405, 1.        , 0.99776798, ..., 0.95765566, 0.96415461,
        0.99394454],
       [0.99845343, 0.99776798, 1.        , ..., 0.95625922, 0.96272006,
        0.99498923],
       ...,
       [0.95776585, 0.95765566, 0.95625922, ..., 1.        , 0.99503819,
        0.97344525],
       [0.96426798, 0.96415461, 0.96272006, ..., 0.99503819, 1.        ,
        0.97344716],
       [0.9946027 , 0.99394454, 0.99498923, ..., 0.97344525, 0.97344716,
        1.        ]])

In [ ]:
g=nx.to_networkx_graph(k)

In [ ]:
def create_node_trace(G):
    # collect node information from G to plot
    node_x = []
    node_y = []
    node_text = []
    node_color = []

    for i, node in enumerate(G.nodes(data=True)):
        # get node x,y position and store
        x, y = node[1]['pos']
        node_x.append(x)
        node_y.append(y)

        node_text.append(node[1]['text'])
        node_color.append(node[1]['color'])

    # create node trace (i.e., scatter plot)
    # make it invisible by default
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=False,
            color=node_color,
            size=10,
            line_width=0.5,
        ),
        text=node_text,
        visible=False
    )

    return node_trace

In [ ]:
def create_edge_trace(G):
    # collect edges information from G to plot
    edge_weight = []
    edge_text = []
    edge_pos = []
    edge_color = []

    for edge in G.edges(data=True):

        # edge is line connecting two points
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_pos.append([[x0, x1, None], [y0, y1, None]])

        # edge line color when drawn
        edge_color.append("black")

    # there is a trace for each edge
    edge_traces = []
    for i in range(len(edge_pos)):

        # edge line width
        line_width = 1

        # is scatter because it is line connecting two points
        trace = go.Scatter(
            x=edge_pos[i][0], y=edge_pos[i][1],
            line=dict(width=line_width, color=edge_color[i]),
            mode='lines',
            visible=False
        )
        edge_traces.append(trace)

    return edge_traces

In [ ]:
def filter_similarity_matrix_at_step(square_matrix, step_value):
    # copy matrix
    aux = square_matrix.copy()

    # set as NaN all values equal to or below threshold value
    aux[aux <= step_value] = np.nan

    # return filtered matrix
    return aux

In [ ]:
def get_interactive_slider_similarity_graph(square_matrix, slider_values, node_text=None, yaxisrange=None, xaxisrange=None):

    # Create figure with plotly
    fig = go.Figure()

    # key: slider value
    # value: list of traces to display for that slider value
    slider_dict = {}

    # total number of traces
    total_n_traces = 0

    # node positions on plot
    #node_pos = None

    # for each possible value in the slider, create and store traces (i.e., plots)
    for i, step_value in enumerate(slider_values):

        # update similarity matrix for the current step
        aux = filter_similarity_matrix_at_step(square_matrix, step_value)

        # create nx graph from sim matrix
        G = nx.to_networkx_graph(aux)

        # remove edges for 0 weight (NaN)
        G.remove_edges_from([(a, b) for a, b, attrs in G.edges(data=True) if np.isnan(attrs["weight"])])

        # assign node positions if None
        node_pos = nx.nx_agraph.graphviz_layout(G)

        # populate nodes with meta information
        for node in G.nodes(data=True):

            # node position
            node[1]['pos'] = node_pos[node[0]]

            # node color
            node[1]['color'] = "orange"

            # node text on hover if any is specified else is empty
            if node_text is not None:
                node[1]['text'] = node_text[node[0]]
            else:
                node[1]['text'] = ""

        # create edge taces (each edge is a trace, thus this is a list)
        edge_traces = create_edge_trace(G)

        # create node trace (a single trace for all nodes, thus it is not a list)
        node_trace = create_node_trace(G)

        # store edge+node traces as single list for the current step value
        slider_dict[step_value] = edge_traces + [node_trace]

        # keep count of the total number of traces
        total_n_traces += len(slider_dict[step_value])

        # make sure that the first slider value is active for visualization
        if i == 0:
            for trace in slider_dict[step_value]:
                # make visible
                trace.visible = True


    # Create steps objects (one step per step_value)
    steps = []
    for step_value in slider_values:

        # count traces before adding new traces
        n_traces_before_adding_new = len(fig.data)

        # add new traces
        fig.add_traces(slider_dict[step_value])

        step = dict(
            # update figure when this step is active
            method="update",
            # make all traces invisible
            args=[{"visible": [False] * total_n_traces}],
            # label on the slider
            label=str(round(step_value, 3)),
        )

        # only toggle this step's traces visible, others remain invisible
        n_traces_for_step_value = len(slider_dict[step_value])
        for i in range(n_traces_before_adding_new, n_traces_before_adding_new + n_traces_for_step_value):
            step["args"][0]["visible"][i] = True

        # store step object in list of many steps
        steps.append(step)

    # create slider with list of step objects
    slider = [dict(
        active=0,
        steps=steps
    )]

    # add slider to figure and create layout
    fig.update_layout(
        sliders=slider,
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(range=xaxisrange, showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(range=yaxisrange, showgrid=False, zeroline=False, showticklabels=False),
        width=700, height=700,
    )

    return fig

In [ ]:
!apt install libgraphviz-dev
!pip install pygraphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk libxdot4
0 upgraded, 8 newly installed, 0 to remove and 24 not upgraded.
Need to get 2,148 kB of archives.
After this operation, 7,427 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libgtk2.0-common all 2.24.32-4ubuntu4 [126 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libgtk2.0-0 amd64 2.24.32-4ubuntu4 [1,791 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libgail18 amd64 2.24.32-4ubuntu4 [14.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libgail-common amd64 2.24.32-4ub

In [ ]:

aux = filter_similarity_matrix_at_step(k,0.5)
G = nx.to_networkx_graph(aux)
# remove edges for 0 weight (NaN)
G.remove_edges_from([(a, b) for a, b, attrs in G.edges(data=True) if np.isnan(attrs["weight"])])

G = nx.drawing.nx_agraph.to_agraph(G)

G.node_attr.update(color="red", style="filled")
G.edge_attr.update(color="blue", width="2.0")

G.draw('/tmp/out.svg', format='svg', prog='neato')



In [ ]:
gr_expand_nodub=pd.read_csv(r"/content/gr_expand_nodub.csv")

In [ ]:
# define slider steps (i.e., threshold values)
slider_steps = np.arange(0.9, 1.05, 0.005)

# get the slider figure
fig = get_interactive_slider_similarity_graph(
    k[40:60,40:60],
    slider_steps,
     node_text=gr_expand_nodub['Molecule'].to_list()[40:60], yaxisrange=None, xaxisrange=None
)
#k[40:60,40:60]
#node_text=gr_expand_nodub['Molecule'].to_list()[40:60]
# [:20,:20] gr_expand_nodub['Molecule'].to_list()[:20]
# plot it
fig.show()
#pyplot.savefig()'/content/n1.svg'
import pickle
pickle.dump(fig, open('/content/n1.fig.pickle', 'wb'))

In [ ]:
from sympy import *
t = symbols('t')
n1 = Function('n1')(t)
a, b, c = symbols('a b c', real=True)
diff_eq = Eq(n1.diff(t), -(a + 2*b*c)*n1 + b*c)
sol = dsolve(diff_eq, n1)
sol

In [ ]:
import numpy as np
np.exp(-0)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the time range
t = np.linspace(0, 100000, 1000)

# Define the first equation


# Define the second equation
y2 = 0.5 * (np.exp(-2.03e-4 * t) + 1)

# Plot the two equations

plt.plot(t, y2, label='0.5 * (exp(-2.03e-4t) + 1)')

# Add labels and legend
plt.xlabel('Time (s)')
plt.ylabel('y')
plt.legend()

# Show the plot
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the time range
t = np.linspace(0, 100000000, 10000)

# Define the first equation
y1 = np.exp(-1.71e-7 * t)

# Define the second equation
y2 = 0.5 * (np.exp(-2.03e-4 * t) + 1)

# Plot the two equations
plt.plot(t, y1, label='exp(-1.71e-7t)')
plt.plot(t, y2, label='0.5 * (exp(-2.03e-4t) + 1)')

# Add labels and legend
plt.xlabel('Time (s)')
plt.ylabel('y')
plt.legend()

# Show the plot
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the time range
t = np.linspace(0, 100000000, 10000)

# Define the first equation
y1 = np.exp(-1.71e-7 * t)



# Plot the two equations
plt.plot(t, y1, label='exp(-1.71e-7t)')


# Add labels and legend
plt.xlabel('Time (s)')
plt.ylabel('y')
plt.legend()

# Show the plot
plt.show()